# Section 3: For-Loops and Iterables
A "for-loop" allows you to iterate over a collection of items, and execute a block of code once for each iteration. The general syntax for a "for-loop" is:

```
for <var> in <iterable>:
    block of code
```

Where `<var>` is any valid variable-identifier and `<iterable>` is any **iterable**.

<div class="alert alert-block alert-info"> 
**Definition**: An **iterable** is any Python object capable of returning its members one at a time. Common examples of iterables include lists, tuples, strings, and dictionaries. Formally, an iterable is any Python object with an `__iter__()` method or with a `__getitem__()` method that implements `Sequence` semantics.
</div>

To be concrete, let's consider the example:
```python
total = 0
for item in [1, 3, 5]:
    total = total + item

print(total)  # `total` has the value 1 + 3 + 5 = 9
# `item` is still defined here, and holds the value 5
```

This code will perform the following steps:
1. Define the variable `total`, and assign it the value `0`
2. Iterate on the list, producing value `1`, define the variable `item` and assign it the value `1`
3. Assign `total` the value `0 + 1`
4. Iterate on the list, producing the value `3` and assigning it to `item`
5. Assign `total` the value `1 + 3`
6. Iterate on the list, producing the value `5` and assigning it to `item`
7. Assign `total` the value `4 + 5`
8. Iterate on the list. A `StopIteration` signal it raised by the list, and the for-loop sequence is exited.
9. Print the value of `total` (9)

Note that the variable `item` will persist after the for-loop block is exited. It will reference the last value from the for-loop iteration (in this case `item` has the value 5). That being said, **you should not write code that depends on the iterate-variable, outside of the context of the for-loop**. In the case that you try to loop over an *empty* iterable, the iterate-variable is never defined:

```python
for x in []:
    print("Hello?")  # this code is never executed
print(x)
```

Because we are attempting to iterate over an empty list, `StopIteration` is raised immediately - before the variable `x` is even defined. Thus the code enclosed within the for-loop is never reached, and the subsequent `print(x)` statement will raise a `NameError`, because `x` was never defined!

Tuple unpacking
https://www.python.org/dev/peps/pep-3132/#specification